##### Local Environment Setup

In [1]:

import os
import sys
# Set JAVA env variable
os.environ["JAVA_HOME"] = r"C:\Program Files\Eclipse Adoptium\jdk-11.0.26.4-hotspot"
# Set Hadoop environment variables 
os.environ['HADOOP_HOME'] = r'C:\hadoop'
os.environ['PATH'] = os.environ['HADOOP_HOME'] + r'\bin;' + os.environ['PATH']
# Set the Python executable path explicitly
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Get Pipeline Tools Path
sys.path.append(R'C:\GitHub\Tools\de')


##### Libraries

In [2]:
import time
import logging
from datetime import datetime

import numpy as np
import pandas as pd

from typing import Dict


from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.types import (StructType, StructField, StringType, 
                            DoubleType, IntegerType, TimestampType, 
                            DateType)
from delta.tables import DeltaTable


In [3]:
from de_pipeline_tools import *

##### Spark Session

In [4]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline")

2025-04-09 10:57:12,032 - INFO - ---Spark session initialized with Delta Lake support---


In [5]:
# Create a database in the Hive warehouse if doesn't exist
spark.sql("CREATE DATABASE IF NOT EXISTS de_pipelines LOCATION 'C:/hive-warehouse/de_pipelines'")

DataFrame[]

##### Inputs/Outputs

In [6]:
data_dir = "../../data"
source_path     = f"{data_dir}/financial_transactions.csv"

source_path  = os.path.abspath(os.path.join(os.getcwd(), *source_path.split('/')))

bronze_table = 'de_pipelines.financial_osb_bronze'
silver_table = 'de_pipelines.financial_osb_silver'
gold_table   = 'de_pipelines.financial_osb_gold'


In [7]:
def get_schema():
    return StructType([
        StructField("transaction_id", StringType(), True),
        StructField("timestamp", StringType(), True), # need to process as StringType and convert to Timestamp
        StructField("customer_id", StringType(), True),
        StructField("account_number", StringType(), True),
        StructField("transaction_type", StringType(), True),
        StructField("amount", DoubleType(), True),
        StructField("currency", StringType(), True),
        StructField("balance_after", DoubleType(), True),
        StructField("status", StringType(), True),
        StructField("merchant", StringType(), True),
        StructField("category", StringType(), True),
        StructField("location", StringType(), True)
    ])

### Define Transformation Functions and Validation Rules

In [8]:
def bronze_transform(df:DataFrame) -> DataFrame:

    # Clean any non-timestamp characters first
    df = df.withColumn(
        "timestamp_cleaned", 
        F.regexp_replace(F.col("timestamp"), "[^0-9\\-: ]", "")
    )

    # Use try_cast to handle invalid timestamps gracefully by returning NULL
    df = df.withColumn(
        "timestamp",
        F.expr("try_cast(timestamp_cleaned as timestamp)")
    )

    # Rename to transaction_date
    df = df.withColumnRenamed('timestamp', 'transaction_timestamp')

    # Drop the intermediate column
    df = df.drop("timestamp_cleaned")

    return df

In [9]:
# Define bronze validation rules
bronze_validation_rules = [
    {
        "name": "has_transaction_id",
        "condition": "transaction_id IS NOT NULL",
        "description": "Transaction ID must be present"
    },
    {
        "name": "valid_amount",
        "condition": "amount IS NULL OR amount > 0",
        "description": "Amount must be positive if not null"
    },
    {
        "name": "valid_transaction_timestamp",
        "condition": "transaction_timestamp IS NULL OR transaction_timestamp <= current_date()",
        "description": "Transaction Timestamp must not be in the future"
    }
]

In [10]:
def silver_transform(df:DataFrame) -> DataFrame:

    # Remove duplicates
    df = df.dropDuplicates(subset=["transaction_id"])

    # Standardize Data
    df = (df
            .withColumn("amount", F.abs(F.col("amount")))
            .withColumn("transaction_type", F.lower(F.col("transaction_type")))
            .withColumn("category", F.lower(F.col("category")))
            .withColumn("status", F.lower(F.col("status")))
    )

    # Filter Data
    # Address bronze layer data validation check concerns
    df = df.filter(
                    (F.col('transaction_id').isNotNull()) # transaction id must exist
                    & (F.col('amount') > 0) # amount must be positive
                    & ((F.col('transaction_timestamp') <= F.current_date()) # must be <= current date
                    |(F.col('transaction_timestamp').isNull()))# or must be Null, no future timestamps
    ) 
    
    # Split timestamp into date and time
    df = (df
            .withColumn("transaction_date", F.to_date("transaction_timestamp"))
            .withColumn("transaction_time", F.date_format("transaction_timestamp", "HH:mm:ss"))
    )
    
    # Derive year_month for partitioning
    df = (df
            .withColumn("year_month", F.date_format(F.col("transaction_date"), "yyyy-MM"))
            .withColumn("processing_timestamp", F.current_timestamp())
    )

    return df

In [11]:
# Define silver validation rules
silver_validation_rules = [
    {
        "name": "valid_transaction_type",
        "condition": "transaction_type IN ('debit', 'credit', 'transfer', 'payment', 'withdrawal', 'deposit') OR transaction_type IS NULL",
        "description": "Transaction type must be one of the valid types"
    },
    {
        "name": "valid_status",
        "condition": "status IN ('completed', 'pending', 'failed', 'cancelled', 'refunded') OR status IS NULL",
        "description": "Status must be one of the valid statuses"
    },
    {
        "name": "valid_currency",
        "condition": "currency IS NULL OR length(currency) = 3",
        "description": "Currency code should be 3 characters if present"
    }
]

In [12]:
def gold_transform(df:DataFrame) -> Dict:
    gold_dfs = {}

    # Gold aggregation 1: Daily summary by category
    daily_category = (df
        .groupBy("transaction_date", "category")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount"),
            F.min("amount").alias("min_amount"),
            F.max("amount").alias("max_amount"),
            F.countDistinct("customer_id").alias("unique_customers")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
    )
    
    gold_dfs["daily_category"] = daily_category
    
    # Gold aggregation 2: Customer summary
    customer_summary = (df
        .groupBy("customer_id")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount"),
            F.min("transaction_date").alias("first_transaction_date"),
            F.max("transaction_date").alias("last_transaction_date"),
            F.approx_count_distinct("category").alias("category_count")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
        .withColumn("days_since_last_transaction", 
                    F.datediff(F.current_date(), F.col("last_transaction_date")))
    )
    
    gold_dfs["customer_summary"] = customer_summary
    
    # Gold aggregation 3: Transaction type summary
    transaction_type_summary = (df
        .groupBy("transaction_type")
        .agg(
            F.count("transaction_id").alias("transaction_count"),
            F.sum("amount").alias("total_amount"),
            F.avg("amount").alias("avg_amount")
        )
        .withColumn("processing_timestamp", F.current_timestamp())
    )
    
    gold_dfs["transaction_type_summary"] = transaction_type_summary

    return gold_dfs

In [13]:
# Define gold validation rules
gold_validation_rules = [
    {
        "name": "positive_transaction_counts",
        "condition": "transaction_count > 0",
        "description": "Transaction counts should be positive"
    },
    {
        "name": "valid_total_amounts",
        "condition": "total_amount >= 0",
        "description": "Total amounts should not be negative"
    }
]

### Running Full Batch Pipeline

In [14]:
run_batch_de_pipeline(spark, 'csv', source_path, get_schema(), 
                        bronze_table, silver_table, gold_table, 
                        bronze_transform, silver_transform, gold_transform, 
                        bronze_validation_rules, silver_validation_rules, gold_validation_rules,
                        pipeline_name='Financial_DE_Pipeline')

2025-04-09 10:57:33,162 - INFO - --Starting data pipeline execution with ID: Financial_DE_Pipeline_20250409_105733--
2025-04-09 10:57:33,163 - INFO - Starting bronze layer processing for c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 10:57:33,853 - INFO - Successfully read CSV data from c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 10:57:34,072 - INFO - Transformation function applied
2025-04-09 10:58:02,925 - INFO - Successfully wrote bronze table: de_pipelines.financial_osb_bronze
2025-04-09 10:58:04,610 - INFO - Running data quality checks for bronze layer
2025-04-09 10:58:05,878 - INFO - Data Quality Metrics for bronze layer:
2025-04-09 10:58:05,881 - INFO -   - Column Count: 15
2025-04-09 10:58:05,883 - INFO -   - Columns: ['transaction_id', 'transaction_timestamp', 'customer_id', 'account_number', 'transaction_type', 'amount', 'currency', 'balance_after', 'status', 'merchant', 'category', 'location', 'ingestion_timestamp', 'source_file', 'ba

{'status': 'success',
 'pipeline_id': 'Financial_DE_Pipeline_20250409_105733',
 'bronze_version': 4,
 'silver_version': 4,
 'timestamp': '2025-04-09T11:00:16.208787',
 'duration_seconds': 163.0434033870697,
 'metrics': {'pipeline_id': 'Financial_DE_Pipeline_20250409_105733',
  'start_time': '2025-04-09T10:57:33.162385',
  'stages': {'bronze': {'duration_seconds': 34.31829905509949,
    'version': 4,
    'status': 'success'},
   'bronze_optimize': {'layer': 'bronze',
    'duration_seconds': 3.5808770656585693,
    'status': 'success'},
   'silver': {'duration_seconds': 22.934001445770264,
    'version': 4,
    'status': 'success',
    'source_bronze_version': 4},
   'silver_optimize': {'layer': 'silver',
    'duration_seconds': 16.801259756088257,
    'status': 'success'},
   'gold': {'duration_seconds': 76.55043959617615,
    'status': 'success',
    'source_silver_version': 4,
    'tables': ['daily_category',
     'customer_summary',
     'transaction_type_summary']},
   'gold_optimiz

### Running Each Layer In Isolation

In [15]:
spark = initialize_local_spark_delta_lake("Financial Data Pipeline - Testing")

2025-04-09 11:00:17,921 - INFO - ---Spark session initialized with Delta Lake support---


#### Bronze

In [16]:
bronzedf, bronze_version = process_batch_bronze_layer(spark, 'csv', source_path, get_schema(), bronze_table,
                               bronze_transform=bronze_transform, validation_rules=bronze_validation_rules,
                               pipeline_id='test', mode='test')

2025-04-09 11:00:19,551 - INFO - Starting bronze layer processing for c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 11:00:19,620 - INFO - Successfully read CSV data from c:\GitHub\DE_Pipelines\data\financial_transactions.csv
2025-04-09 11:00:19,662 - INFO - Transformation function applied
2025-04-09 11:00:19,663 - WARNING - --- Bronze layer in Test Mode ---
2025-04-09 11:00:19,664 - INFO - Running data quality checks for bronze layer
2025-04-09 11:00:20,385 - INFO - Data Quality Metrics for bronze layer:
2025-04-09 11:00:20,387 - INFO -   - Column Count: 15
2025-04-09 11:00:20,388 - INFO -   - Columns: ['transaction_id', 'transaction_timestamp', 'customer_id', 'account_number', 'transaction_type', 'amount', 'currency', 'balance_after', 'status', 'merchant', 'category', 'location', 'ingestion_timestamp', 'source_file', 'batch_id']
2025-04-09 11:00:20,394 - INFO -   - Sample size: 169
2025-04-09 11:00:20,397 - WARNING -   - Columns with high null percentages: {'mercha

In [17]:
bronzedf.show(5)

+--------------+---------------------+-----------+--------------+----------------+-------+--------+-------------+---------+----------+---------+----------------+--------------------+--------------------+--------+
|transaction_id|transaction_timestamp|customer_id|account_number|transaction_type| amount|currency|balance_after|   status|  merchant| category|        location| ingestion_timestamp|         source_file|batch_id|
+--------------+---------------------+-----------+--------------+----------------+-------+--------+-------------+---------+----------+---------+----------------+--------------------+--------------------+--------+
|   TXN00000001|  2023-06-05 11:37:36| CUST001040| ACCT-86152351|          refund| 598.65|     USD|      2900.03| disputed|Mastercard|education|      Denver, CO|2025-04-09 11:00:...|file:///c:/GitHub...|    test|
|   TXN00000002|  2023-06-04 15:42:51|       NULL| ACCT-52201156|        transfer|-908.13|     USD|      5944.76|completed|      NULL|     NULL|   S

#### Silver

In [18]:
silverdf, silver_version = process_batch_silver_layer(spark, bronze_table, silver_table, 
                                                    silver_transform=silver_transform, validation_rules=silver_validation_rules,
                                                    pipeline_id='test', mode='test', bronze_version=None)

2025-04-09 11:00:21,461 - INFO - Starting silver layer processing
2025-04-09 11:00:22,385 - INFO - Successfully read bronze data version 4
2025-04-09 11:00:22,551 - INFO - Transformation function applied
2025-04-09 11:00:22,553 - WARNING - --- Silver layer in Test Mode ---
2025-04-09 11:00:22,554 - INFO - Running data quality checks for silver layer
2025-04-09 11:00:29,847 - INFO - Data Quality Metrics for silver layer:
2025-04-09 11:00:29,849 - INFO -   - Column Count: 19
2025-04-09 11:00:29,850 - INFO -   - Columns: ['transaction_id', 'transaction_timestamp', 'customer_id', 'account_number', 'transaction_type', 'amount', 'currency', 'balance_after', 'status', 'merchant', 'category', 'location', 'ingestion_timestamp', 'source_file', 'batch_id', 'transaction_date', 'transaction_time', 'year_month', 'processing_timestamp']
2025-04-09 11:00:29,853 - INFO -   - Sample size: 135
2025-04-09 11:00:29,854 - WARNING -   - Columns with high null percentages: {'merchant': 43.7037037037037, 'cate

In [19]:
silverdf.show(5)

+--------------+---------------------+-----------+--------------+----------------+-------+--------+-------------+---------+-----------+-------------+----------------+--------------------+--------------------+--------------------+----------------+----------------+----------+--------------------+
|transaction_id|transaction_timestamp|customer_id|account_number|transaction_type| amount|currency|balance_after|   status|   merchant|     category|        location| ingestion_timestamp|         source_file|            batch_id|transaction_date|transaction_time|year_month|processing_timestamp|
+--------------+---------------------+-----------+--------------+----------------+-------+--------+-------------+---------+-----------+-------------+----------------+--------------------+--------------------+--------------------+----------------+----------------+----------+--------------------+
|   TXN00000001|  2023-06-05 11:37:36| CUST001040| ACCT-86152351|          refund| 598.65|     USD|      2900.03

##### Investigating Silver Validation Check Failures:
Corrupt "status" column

In [20]:
silverdf.groupBy('status').count().show()

+---------+-----+
|   status|count|
+---------+-----+
|  pendlng|    1|
|completed|  264|
| dioputed|    1|
| dirputed|    1|
|completpd|    1|
|   failed|  285|
|     NULL|   13|
|completei|    1|
|  pjnding|    1|
| oeversed|    1|
|  sending|    1|
| disiuted|    1|
|  pendinz|    1|
| repersed|    1|
| disptted|    1|
|  pendiny|    1|
|complethd|    1|
|  pending|  269|
|   faihed|    1|
|  pedding|    1|
+---------+-----+
only showing top 20 rows



#### Gold

In [21]:
gold_dfs = process_batch_gold_layer(spark, silver_table, gold_table, gold_transform, 
                                    validation_rules=gold_validation_rules, pipeline_id='test', mode='test', silver_version=None)

2025-04-09 11:00:36,403 - INFO - Starting gold layer processing
2025-04-09 11:00:36,945 - INFO - Successfully read silver data version 5
2025-04-09 11:00:37,104 - INFO - Transformation function applied
2025-04-09 11:00:37,105 - WARNING - Gold layer in Test Mode
2025-04-09 11:00:37,108 - INFO - Running data quality checks for de_pipelines.financial_osb_gold_daily_category layer
2025-04-09 11:00:46,714 - INFO - Data Quality Metrics for de_pipelines.financial_osb_gold_daily_category layer:
2025-04-09 11:00:46,716 - INFO -   - Column Count: 9
2025-04-09 11:00:46,719 - INFO -   - Columns: ['transaction_date', 'category', 'transaction_count', 'total_amount', 'avg_amount', 'min_amount', 'max_amount', 'unique_customers', 'processing_timestamp']
2025-04-09 11:00:46,720 - INFO -   - Sample size: 197
2025-04-09 11:00:46,722 - WARNING -   - Columns with high null percentages: {'category': 30.456852791878177}
2025-04-09 11:00:46,724 - INFO - Validating dataframe with 2 rules
2025-04-09 11:00:53,303

In [22]:
for table_name in gold_dfs.keys():
    gold_dfs[table_name].show(5)

+----------------+--------+-----------------+------------------+-----------------+----------+----------+----------------+--------------------+
|transaction_date|category|transaction_count|      total_amount|       avg_amount|min_amount|max_amount|unique_customers|processing_timestamp|
+----------------+--------+-----------------+------------------+-----------------+----------+----------+----------------+--------------------+
|      2023-05-25|    fees|                1|             18.07|            18.07|     18.07|     18.07|               1|2025-04-09 11:01:...|
|      2023-12-05|  income|                1|           1818.85|          1818.85|   1818.85|   1818.85|               1|2025-04-09 11:01:...|
|      2023-10-01|    fees|                1|             32.44|            32.44|     32.44|     32.44|               1|2025-04-09 11:01:...|
|      2023-12-27|  dining|                1|           1307.27|          1307.27|   1307.27|   1307.27|               1|2025-04-09 11:01:...|